In [22]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
health = pd.read_csv('data/massivenew_cleaned_trainingdata.csv')
original = pd.read_csv('data/2020_Competition_Training.csv')
health = health.set_index('person_id_syn')
original = original.set_index('person_id_syn')

/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (80,193) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
health['rx_overall_pmpm_ct'] = original['rx_overall_pmpm_ct']
health['rx_mail_pmpm_ct'] =original['rx_mail_pmpm_ct']
health['submcc_men_depr_pmpm_ct']=original['submcc_men_depr_pmpm_ct']
health['rx_gpi2_17_pmpm_ct']=original['rx_gpi2_17_pmpm_ct']
health['submcc_ben_othr_pmpm_ct']=original['submcc_ben_othr_pmpm_ct']
health['rx_gpi2_39_pmpm_ct']=original['rx_gpi2_39_pmpm_ct']
health['submcc_sor_eye_pmpm_ct']=original['submcc_sor_eye_pmpm_ct']
health['rx_branded_pmpm_ct']=original['rx_branded_pmpm_ct']
health['submcc_men_abus_pmpm_ct']=original['submcc_men_abus_pmpm_ct']
health['rx_gpi2_43_pmpm_ct']=original['rx_gpi2_43_pmpm_ct']
health['rx_gpi2_49_pmpm_ct']=original['rx_gpi2_49_pmpm_ct']
health['submcc_vco_vac_pmpm_ct']=original['submcc_vco_vac_pmpm_ct']
health['rx_gpi2_75_pmpm_ct']=original['rx_gpi2_75_pmpm_ct']
health['submcc_rsk_smok_pmpm_ct']=original['submcc_rsk_smok_pmpm_ct']
health['mabh_seg'] = original['mabh_seg']
health = pd.get_dummies(health)

In [29]:
group1 = list(health.columns[800:])
group3 = ['Credit_Stress_clusters','cms_risk_ma_nbr_rx_combined','med_ambulance_visit_ct_pmpm',
'est_age','cms_low_income_ind','cons_n2pmv','count_vco','cms_disabled_ind','submcc_ner_othr_ind',
'credit_hh_dpd_average', 'cms_risk_adjustment_factor_a_amt_rank','betos_o1a_ind','StressIndex',
'credit_num_totalallcredit_collections','submcc_ben_othr_ind','betos_m5d_pmpm_ct_rank','cons_n2mob',
'PCA18_submcc','betos_y2_ind','betos_m5b_pmpm_ct_rank', 'reverse_raf', 'hedis_dia_hba1c_test', 'count_bld_rank',
'cons_homstat_Renter', 'anomaly_score', 'mabh_seg_H2', 'betos_m5c_pmpm_ct + betos_o1g_pmpm_ct', 
'total_physician_office_visit_ct_pmpm', 'rx_gpi2_90_pmpm_ct_rank', 'PCA4_creditbal', 'count_mus',
'rx_gpi2_17_pmpm_ct_rank', 'betos_o1a_pmpm_ct * cons_n2pmv', 'total_ambulance_visit_ct_pmpm + total_er_visit_ct_pmpm',
'cms_partd_ra_factor_amt * cms_tot_partd_payment_amt', 'total_ambulance_visit_ct_pmpm',
'med_physician_office_visit_ct_pmpm', 'cons_retail_buyer', 'cons_hhcomp_Min Two People, No Children',
'count_ner_rank', 'total_emer_visits', 'credit_hh_nonmtgcredit_60dpd_rank', 'count_can', 'PCA3_rx', 
'cons_hhcomp_One Person, Children', 'med_outpatient_visit_ct_pmpm', 'cms_ma_risk_score_nbr + cms_tot_partd_payment_amt',
'fci_score', 'cci_score', 'hcc_weighted_sum','rx_overall_pmpm_ct','rx_mail_pmpm_ct','submcc_men_depr_pmpm_ct',
'rx_gpi2_17_pmpm_ct','submcc_ben_othr_pmpm_ct','rx_gpi2_39_pmpm_ct','submcc_sor_eye_pmpm_ct',
'rx_branded_pmpm_ct','submcc_men_abus_pmpm_ct','rx_gpi2_43_pmpm_ct','rx_gpi2_49_pmpm_ct',
'submcc_vco_vac_pmpm_ct','rx_gpi2_75_pmpm_ct','submcc_rsk_smok_pmpm_ct', 'credit_hh_totalallcredit_severederog_rank',
'ccsp_239_ind']
grouped = group1 + group3
grouped = list(set(group1 + group3))

In [30]:
Y = health['transportation_issues']
X = health[grouped]

In [31]:
from sklearn.model_selection import train_test_split

X_train82, X_holdout_test, y_train82, y_holdout_test = train_test_split(X, Y, test_size=0.25, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train82, y_train82, test_size=0.2)

In [4]:
def ForwardSelection(train_features, train_labels, test_features, test_labels):
    '''
    Completes a forward selection iterative model, starting with no features and adding a new feature
    based on the best ROC AUC score. This program, for each iteration, will print the best feature that was 
    added to the model and it's associated ROC AUC score.
    
    Takes 4 arguments: 
    
    train_features: training dataset with no labels
    train_labels: training dataset with only target labels
    test_features: test dataset with no labels
    test_labels: test dataset with only target variables
    '''
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    
    best_roc = 0
    best_roc_list = [0]
    checks_before_break = 0
    features = list(train_features.columns)
    best_features = []
    model = XGBClassifier(n_estimators=100,
                      max_depth = 2,
                      scale_pos_weight = 5, 
                      subsample = 0.6,
                      colsample_bytree = 0.8,
                      min_child_weight = 5,
                      eta = 0.2,
                      alpha = 0.01,
                      gamma = 2, 
                      reg_lambda = 4.5)
    
    while (len(features) >= 2): 
        for feature in features: 
            if best_features == []:
                model.fit(np.array(train_features[feature]).reshape(-1,1), train_labels)
                p_pred = model.predict_proba(np.array(test_features[feature]).reshape(-1,1))
                p_pred = p_pred[:,1]
                roc_value = roc_auc_score(test_labels, p_pred)
            else:
                features_to_test = best_features + [feature]
                model.fit(train_features[features_to_test], train_labels)
                p_pred = model.predict_proba(test_features[features_to_test])
                p_pred = p_pred[:,1]
                roc_value = roc_auc_score(test_labels, p_pred)

            if roc_value > best_roc:
                best_roc = roc_value
                current_best = feature 

        best_features.append(current_best)
        best_roc_list.append(best_roc)

        features.remove(current_best)
        print(f'Added Feature: \'{current_best}\', ROC: {best_roc}')
    return best_features, best_roc_list 

In [5]:
best_features, best_roc = ForwardSelection(X_train, y_train, X_test, y_test)

Added Feature: 'Credit_Stress_clusters', ROC: 0.6600241299664548
Added Feature: 'cms_risk_ma_nbr_rx_combined', ROC: 0.6913004549428685
Added Feature: 'med_ambulance_visit_ct_pmpm', ROC: 0.7073787643885249
Added Feature: 'est_age', ROC: 0.7137695933170645
Added Feature: 'cms_low_income_ind', ROC: 0.7212424147872241
Added Feature: 'cons_n2pmv', ROC: 0.7231081917195493
Added Feature: 'count_vco', ROC: 0.7260103472392736
Added Feature: 'cms_disabled_ind', ROC: 0.7291339982111943
Added Feature: 'submcc_ner_othr_ind', ROC: 0.7309633209145188
Added Feature: 'credit_hh_dpd_average', ROC: 0.7317960802876509
Added Feature: 'cms_risk_adjustment_factor_a_amt_rank', ROC: 0.7323732168452666
Added Feature: 'betos_o1a_ind', ROC: 0.7339629093178338
Added Feature: 'StressIndex', ROC: 0.7341824103137324
Added Feature: 'credit_num_totalallcredit_collections', ROC: 0.736051696011767
Added Feature: 'submcc_ben_othr_ind', ROC: 0.7366703837422559
Added Feature: 'betos_m5d_pmpm_ct_rank', ROC: 0.736709460311944

ValueError: list.remove(x): x not in list

In [34]:
from sklearn.metrics import roc_auc_score
train_features, train_labels, test_features, test_labels = X_train, y_train, X_test, y_test
best_roc = 0
best_roc_list = [0]
checks_before_break = 0
features = list(X_train.columns)
best_features = ['Credit_Stress_clusters','cms_risk_ma_nbr_rx_combined','med_ambulance_visit_ct_pmpm',
'est_age','cms_low_income_ind','cons_n2pmv','count_vco','cms_disabled_ind','submcc_ner_othr_ind',
'credit_hh_dpd_average', 'cms_risk_adjustment_factor_a_amt_rank','betos_o1a_ind','StressIndex',
'credit_num_totalallcredit_collections','submcc_ben_othr_ind','betos_m5d_pmpm_ct_rank','cons_n2mob',
'PCA18_submcc','betos_y2_ind','betos_m5b_pmpm_ct_rank', 'reverse_raf', 'hedis_dia_hba1c_test', 'count_bld_rank',
'cons_homstat_Renter', 'anomaly_score', 'mabh_seg_H2', 'betos_m5c_pmpm_ct + betos_o1g_pmpm_ct', 
'total_physician_office_visit_ct_pmpm', 'rx_gpi2_90_pmpm_ct_rank', 'PCA4_creditbal', 'count_mus',
'rx_gpi2_17_pmpm_ct_rank', 'betos_o1a_pmpm_ct * cons_n2pmv', 'total_ambulance_visit_ct_pmpm + total_er_visit_ct_pmpm',
'cms_partd_ra_factor_amt * cms_tot_partd_payment_amt', 'total_ambulance_visit_ct_pmpm',
'med_physician_office_visit_ct_pmpm', 'cons_retail_buyer', 'cons_hhcomp_Min Two People, No Children',
'count_ner_rank', 'total_emer_visits', 'credit_hh_nonmtgcredit_60dpd_rank', 'count_can', 'PCA3_rx', 
'cons_hhcomp_One Person, Children', 'med_outpatient_visit_ct_pmpm', 'cms_ma_risk_score_nbr + cms_tot_partd_payment_amt',
'fci_score', 'cci_score', 'hcc_weighted_sum','rx_overall_pmpm_ct','rx_mail_pmpm_ct','submcc_men_depr_pmpm_ct',
'rx_gpi2_17_pmpm_ct','submcc_ben_othr_pmpm_ct','rx_gpi2_39_pmpm_ct','submcc_sor_eye_pmpm_ct',
'rx_branded_pmpm_ct','submcc_men_abus_pmpm_ct','rx_gpi2_43_pmpm_ct','rx_gpi2_49_pmpm_ct',
'submcc_vco_vac_pmpm_ct','rx_gpi2_75_pmpm_ct','submcc_rsk_smok_pmpm_ct', 'credit_hh_totalallcredit_severederog_rank',
'ccsp_239_ind', 'anomaly_decision']
i = 0
baseline = 0.735
features = [x for x in features if x not in best_features]
model = XGBClassifier(n_estimators=100,
                      max_depth = 2,
                      scale_pos_weight = 5.5, 
                      subsample = 0.6,
                      colsample_bytree = 0.8,
                      min_child_weight = 5,
                      eta = 0.2,
                      alpha = 0.01,
                      gamma = 2, 
                      reg_lambda = 4.5)
    
while (i != 10): 
    for feature in features: 
        features_to_test = best_features + [feature]
        model.fit(train_features[features_to_test], train_labels)
        p_pred = model.predict_proba(test_features[features_to_test])
        p_pred = p_pred[:,1]
        roc_value = roc_auc_score(test_labels, p_pred)

        if roc_value > best_roc:
            i = 0
            best_roc = roc_value
            current_best = feature
            print(current_best, best_roc)
        if roc_value <= baseline:
            features.remove(feature)

    if current_best == best_features[-1]:
        i += 1
        continue
    best_features.append(current_best)
    best_roc_list.append(best_roc)

    features.remove(current_best)
    print(f'Added Feature: \'{current_best}\', ROC: {best_roc}, {len(features)}')

cons_n2pmv * submcc_ner_migr_pmpm_ct 0.7400670403830658
credit_num_nonagn1stmorg_bankruptc + credit_num_totalallcredit_collections 0.7401657317299944
cons_n2pmv * credit_num_consumerfinance_new 0.7409309528605964
credit_hh_1stmtg_severederog + credit_hh_studentloan 0.7410345836174301
credit_bal_agencyfirstmtg_new + credit_num_autobank_new 0.7411237835492469
credit_hh_autofinance * credit_num_totalallcredit_collections 0.7412000054236658
credit_hh_bankcard_severederog + credit_num_autofinance 0.7412173417839212
credit_hh_bankcard_severederog + credit_num_autobank 0.7420208191284944
Added Feature: 'credit_hh_bankcard_severederog + credit_num_autobank', ROC: 0.7420208191284944, 6452
credit_hh_agencyfirstmtg * credit_num_mtg_collections 0.7422032867414616
fci_score * weighted_three_scores 0.7422200419946693
Credit_All_clusters * MobilityIndex 0.7426730149383252
credit_minmob_1stmtgcredit * credit_num_1stmtg_severederog 0.7430828891092791
cms_tot_ma_payment_amt * cons_n2pmv 0.74318932855017

KeyboardInterrupt: 